In [1]:
import torch
import cv2
from matplotlib import pyplot as plt
from numpy import ndarray as nd
from ultralytics import YOLO
import pandas as pd
import os
from PIL import Image


In [2]:

# 모델 로드
model = YOLO('./best.pt')

In [3]:
# 웹캠 열기
cap = cv2.VideoCapture(1)

In [ ]:

while cap.isOpened():
    ret, frame = cap.read()
    
    results = model(frame)
    names = model.predict(frame)
    print(names.name)
    cv2.imshow('Img-Detection', nd.squeeze(results[0].plot()))
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [3]:

folder_path = "./FruitsTest/"

image_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.jpeg', '.gif'))]

results = []
for image_file in image_files:
    with Image.open(image_file) as img:
        print(os.path.basename(image_file))
    results.append(model(image_file))

    
    res = model(image_file,conf=0.5)
    names = model.names
    
    for r in res:
        print(r.boxes.cls)
        for c in r.boxes.cls:
            print(names[int(c)])

    res_plotted = res[0].plot()

        
    resized_img = cv2.resize(res_plotted, (800, 600))
    cv2.imshow("Image_detections", resized_img)
    
    ESC = cv2.waitKey(0)
    if ESC == 27:
        cv2.destroyAllWindows()
        exit()

0_1.png



image 1/1 /Users/sanghee/Desktop/LG_Internship/YOLO/FruitsTest/0_1.png: 640x576 2 Apples, 80.5ms
Speed: 2.3ms preprocess, 80.5ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 576)

image 1/1 /Users/sanghee/Desktop/LG_Internship/YOLO/FruitsTest/0_1.png: 640x576 (no detections), 82.7ms
Speed: 2.0ms preprocess, 82.7ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 576)

image 1/1 /Users/sanghee/Desktop/LG_Internship/YOLO/FruitsTest/Unknown.jpeg: 448x640 1 Apple, 2 Apricots, 2 Beetroots, 65.0ms
Speed: 1.4ms preprocess, 65.0ms inference, 0.6ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /Users/sanghee/Desktop/LG_Internship/YOLO/FruitsTest/Unknown.jpeg: 448x640 1 Apple, 1 Apricot, 1 Beetroot, 64.8ms
Speed: 1.4ms preprocess, 64.8ms inference, 0.4ms postprocess per image at shape (1, 3, 448, 640)


Unknown.jpeg
tensor(0.)
Apple
tensor(1.)
Apricot
tensor(6.)
Beetroot



image 1/1 /Users/sanghee/Desktop/LG_Internship/YOLO/FruitsTest/0_5.png: 640x576 1 Apple, 87.6ms
Speed: 3.6ms preprocess, 87.6ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 576)

image 1/1 /Users/sanghee/Desktop/LG_Internship/YOLO/FruitsTest/0_5.png: 640x576 1 Apple, 66.3ms
Speed: 1.5ms preprocess, 66.3ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 576)


0_5.png
tensor(0.)
Apple


: 